In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from pathlib import Path
import os
import json
from core.models.model_factory import create_model
from core.data.dataset import EmotionDataset
from core.training.trainer import train_model
import numpy as np

import mlflow
import optuna
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR, ReduceLROnPlateau


def objective(trial: optuna.Trial):
    """Optuna가 최적화할 목표 함수"""
    
    # MLflow 실험 시작
    with mlflow.start_run():
        
        # --- 1. 하이퍼파라미터 제안 ---
        # Optuna가 이 범위 내에서 최적의 값을 찾아 제안합니다.
        lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
        optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW"])
        scheduler_name = trial.suggest_categorical("scheduler", ["StepLR", "CosineAnnealingLR", "ReduceLROnPlateau"])
        
        # MLflow에 제안된 하이퍼파라미터 기록
        mlflow.log_params(trial.params)
        
        # --- 2. 데이터 및 모델 준비 ---
        
        # CUDA 성능 플래그 최적화
        torch.backends.cudnn.benchmark = True
        # TF32 텐서 코어 사용을 허용하여 Ampere 아키텍처 이상 GPU에서 연산 속도 향상
        torch.backends.cuda.matmul.allow_tf32 = True

        DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {DEVICE}")
    
        #MODEL_NAME = 'resnet18'            
        #MODEL_NAME = 'resnet50' 
        #MODEL_NAME = 'mobilenet_v3_small'  
        #MODEL_NAME = 'shufflenet_v2'       
        #MODEL_NAME = 'efficientnet_v2_s'   
        #MODEL_NAME = 'squeezenet'          
        #MODEL_NAME = 'emotionnet'  # 감정 인식 전용 모델
        MODEL_NAME = 'emonet'       # 경량화된 감정 인식 모델
        
        sampling_percent = "1"

        MISCLASSIFIED_DIR = Path(f"./datasets/misclassified_images/{MODEL_NAME}_{sampling_percent}") # 오답 이미지를 저장할 폴더 경로 정의
        DATA_DIR = Path(f"./datasets/KECV_{sampling_percent}")
        # 대쉬보드에 사용된 데이터셋을 표시, 상세화면의 Tags와 Parameters에 기록
        mlflow.log_param("dataset_path", str(DATA_DIR))
        mlflow.set_tag("dataset_description", "테스트용 1% 데이터")
        NUM_CLASSES = 7  # 데이터셋의 클래스 수에 맞게 조정해야 합니다. ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
        BATCH_SIZE = 64  # 배치 크기를 늘려 GPU 메모리 사용 최적화
        LEARNING_RATE = 0.001
        NUM_EPOCHS = 5
        EARLY_STOPPING_PATIENCE = 10 # 10번 연속 성능 개선이 없으면 조기 종료
        STEPS_PER_EPOCH = None # 빠른 테스트를 위해 에폭당 배치 수를 제한하려면 숫자로 변경 (예: 100)
        train_transform = None
        val_transform = None
        #scheduler_name = 'CosineAnnealingLR'  # CosineAnnealingLR로 스케줄러 설정
        
        if MODEL_NAME == 'emotionnet':
            # 48x48 크기, 흑백(Grayscale), 정규화
            # RandomResizedCrop + TrivialAugmentWide (강력한 데이터 증강 방법)
            train_transform = transforms.Compose([
                #transforms.Resize((48, 48)),
                # 원본 이미지의 80% ~ 100% 사이를 무작위로 잘라 48x48 크기로 만듦
                transforms.RandomResizedCrop(size=48, scale=(0.8, 1.0)),
                # 잘라낸 이미지에 최적의 증강 정책을 자동으로 적용
                transforms.TrivialAugmentWide(),
                # 흑백으로 변환
                transforms.Grayscale(num_output_channels=1),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5], std=[0.5]) # 흑백 이미지 정규화
            ])
            val_transform = transforms.Compose([
                transforms.Resize((48, 48)),
                transforms.Grayscale(num_output_channels=1),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5], std=[0.5]) # 흑백 이미지는 채널이 1개
            ])

        elif MODEL_NAME == 'emonet':
            # 데이터 증강을 포함한 훈련용 Transform 정의
            train_transform = transforms.Compose([
                #transforms.Resize((256, 256)),
                transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
                transforms.TrivialAugmentWide(), 
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
            # 증강이 없는 검증/테스트용 Transform 정의
            val_transform = transforms.Compose([
                transforms.Resize((256, 256)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
            
        else:
            # 데이터 증강을 포함한 훈련용 Transform 정의
            train_transform = transforms.Compose([
                #transforms.Resize((224, 224)),
                transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
                transforms.TrivialAugmentWide(), 
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
            ])
            # 증강이 없는 검증/테스트용 Transform 정의
            val_transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
            ])
            
        # 훈련용과 검증용 데이터셋을 각각 생성.
        train_dataset = EmotionDataset(data_dir=DATA_DIR / "train", transform=train_transform)
        val_dataset = EmotionDataset(data_dir=DATA_DIR / "val", transform=val_transform)

        # DataLoader I/O 튜닝
        train_loader = DataLoader(
            train_dataset, 
            batch_size=BATCH_SIZE, 
            shuffle=True,
            # CPU 코어를 최대한 활용하여 데이터를 미리 GPU 메모리로 올리는 작업을 병렬 처리
            num_workers=min(8, os.cpu_count()), 
            pin_memory=True, # GPU로의 데이터 전송 속도 향상
            persistent_workers=True, # 워커 프로세스를 계속 유지하여 오버헤드 감소
            prefetch_factor=2, # 각 워커가 미리 로드할 배치 수
            drop_last=True # 마지막 배치가 배치 사이즈보다 작을 경우 버려서 연산 일관성 유지
        )
        val_loader = DataLoader(
            val_dataset, 
            batch_size=BATCH_SIZE, 
            shuffle=False,
            num_workers=min(8, os.cpu_count()),
            pin_memory=True,
            persistent_workers=True,
            prefetch_factor=2
        )

        NUM_CLASSES = len(train_dataset.classes)
        
        print("데이터 준비 완료!")
        print(f"훈련 데이터셋 크기: {len(train_dataset)}")
        print(f"클래스 수: {NUM_CLASSES} -> {train_dataset.classes}")

        # 모델, 손실 함수, 옵티마이저 준비
        model = create_model(model_name=MODEL_NAME, num_classes=NUM_CLASSES)
        model.to(DEVICE)
        
        optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
        if scheduler_name == 'StepLR':
            scheduler = StepLR(optimizer, step_size=7, gamma=0.1)
        elif scheduler_name == 'ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)
        else:
            scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-6)
        
        criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
        
        print(f"'{MODEL_NAME}' 모델, 손실 함수, 옵티마이저 준비 완료!")
        
        CHECKPOINT_PATH = f'./infrastructure/models/weights/checkpoints/{MODEL_NAME}_{sampling_percent}_trained.pth'
        if os.path.exists(CHECKPOINT_PATH):
            print("체크포인트를 불러옵니다...")
            checkpoint = torch.load(CHECKPOINT_PATH)
            model.load_state_dict(checkpoint)
            print("체크포인트(모델 가중치) 로드 완료!")
        else:
            print("체크포인트가 존재하지 않습니다. 처음부터 훈련을 시작합니다.")
        
        
    
        # --- 3. 모델 훈련 ---
        # trainer가 이제 최고 점수(best_metrics)만 반환하도록 수정했다고 가정
        #best_metrics = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, DEVICE, num_epochs=NUM_EPOCHS)
        trained_model, saved_metrics = train_model(model, 
                                    train_loader, 
                                    val_loader, 
                                    criterion, 
                                    optimizer, 
                                    scheduler,
                                    DEVICE, 
                                    num_epochs=NUM_EPOCHS,
                                    patience=EARLY_STOPPING_PATIENCE,
                                    steps_per_epoch=STEPS_PER_EPOCH,
                                    misclassified_dir=MISCLASSIFIED_DIR
                                    )
        
        # --- 4. MLflow에 결과 기록 ---
        # 최고 검증 손실과 정확도, F1 Score 등을 기록
        mlflow.log_metrics({
            "best_train_loss": float(saved_metrics['train_loss']),
            "best_train_accuracy": float(saved_metrics['train_accuracy']),
            "best_val_loss": float(saved_metrics['val_loss']),
            "best_val_accuracy": float(saved_metrics['val_accuracy']),
            "best_macro_f1": float(saved_metrics['macro_f1_score']),
        })
        
        # 훈련된 모델 저장 (옵션)
        CHECKPOINT_PATH = f'./infrastructure/models/weights/checkpoints/{MODEL_NAME}_{sampling_percent}_trained.pth'
        torch.save(trained_model.state_dict(), CHECKPOINT_PATH)
        
        # MLflow에 모델 저장
        #input_example = np.random.rand(1, 3, 256, 256)
        #mlflow.pytorch.log_model(trained_model, name="EMONET_1P", input_example=input_example, pip_requirements="pip_requirements.txt")
        mlflow.pytorch.log_model(trained_model, name="EMONET_1P", pip_requirements="pip_requirements.txt")
        print("훈련된 모델 가중치가 저장되었습니다.")

        # 최고 성능 시점의 상세 분석 결과를 JSON으로 저장
        METRICS_PATH = f'./infrastructure/models/weights/checkpoints/{MODEL_NAME}_{sampling_percent}_percent_trained_metrics.json'
        with open(METRICS_PATH, 'a', encoding='utf-8') as f:
            json.dump(saved_metrics, f, ensure_ascii=False, indent=4)
        print(f"상세 분석 결과가 저장되었습니다: {METRICS_PATH}")
                
        # --- 5. Optuna에 목표값 반환 ---
        # 우리는 검증 손실(val_loss)을 최소화하는 것을 목표로 함
        return float(saved_metrics['val_loss'])

if __name__ == '__main__':
    #코드 실행 전 아래 명령어를 터미널에서 실행
    # mlflow ui
    # 위의 명령어만 먼저해보고 에러 MlflowException: When an mlflow-artifacts URI was supplied, the tracking URI must be a valid http or https URI 가 발생하면 아래 명령어 실행.
    # mlflow server --host 127.0.0.1 --port 5000
    
    # MLflow 추적 서버 URI 설정
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    
    # MLflow 실험 이름 설정, 대쉬보드에서 훈련을 구분하여 보기위해 사용.
    mlflow.set_experiment("Emotion Classification Tuning")

    # Optuna Study 생성: 'minimize' 방향으로 objective 함수를 최적화
    study = optuna.create_study(direction="minimize")
    
    # 100번의 다른 하이퍼파라미터 조합으로 실험(Trial) 실행
    study.optimize(objective, n_trials=5)
    
    print("Hyperparameter optimization finished.")
    print("Best trial:")
    trial = study.best_trial
    
    print(f"  Value (Best Val Loss): {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")


[I 2025-08-23 00:16:08,634] A new study created in memory with name: no-name-66cc9591-61c2-4480-b605-23d618a36bb3


Using device: cuda
데이터 준비 완료!
훈련 데이터셋 크기: 2786
클래스 수: 7 -> ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
사전 훈련된 EmoNet 가중치를 불러옵니다 (Fine-tuning)...
'emonet' 모델, 손실 함수, 옵티마이저 준비 완료!
체크포인트를 불러옵니다...
체크포인트(모델 가중치) 로드 완료!
Epoch 1/5
----------
  [Batch 20/43] Train Loss: 1.3715 Acc: 0.6875
  [Batch 43/43] Train Loss: 1.1940 Acc: 0.6875
Train Loss: 1.3166 Acc: 0.6635
Val Loss: 1.3299 Acc: 0.7127 Macro-F1: 0.5061
  -> Val Loss 개선됨! (1.3299) 모델 저장.
Epoch 2/5
----------
  [Batch 20/43] Train Loss: 1.3329 Acc: 0.6875
  [Batch 43/43] Train Loss: 1.2094 Acc: 0.7031
Train Loss: 1.3021 Acc: 0.6846
Val Loss: 1.3250 Acc: 0.7164 Macro-F1: 0.5074
  -> Val Loss 개선됨! (1.3250) 모델 저장.
Epoch 3/5
----------
  [Batch 20/43] Train Loss: 1.2890 Acc: 0.6406
  [Batch 43/43] Train Loss: 1.2328 Acc: 0.6562
Train Loss: 1.2909 Acc: 0.6668
Val Loss: 1.3001 Acc: 0.7127 Macro-F1: 0.5057
  -> Val Loss 개선됨! (1.3001) 모델 저장.
Epoch 4/5
----------
  [Batch 20/43] Train Loss: 1.2412 Acc: 0.6406
  [Batch 43/43] Train Loss: 1.1320 Ac

2025/08/23 00:18:48 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/08/23 00:18:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


훈련된 모델 가중치가 저장되었습니다.
상세 분석 결과가 저장되었습니다: ./infrastructure/models/weights/checkpoints/emonet_1_percent_trained_metrics.json
🏃 View run abundant-stag-438 at: http://127.0.0.1:5000/#/experiments/957529077469630842/runs/26fb18654f394386bc6053a10c3cc6b9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/957529077469630842


[I 2025-08-23 00:18:51,548] Trial 0 finished with value: 1.3001 and parameters: {'lr': 1.7344282965791717e-05, 'optimizer': 'Adam', 'scheduler': 'StepLR'}. Best is trial 0 with value: 1.3001.


...

2025/08/23 00:29:18 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/08/23 00:29:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


훈련된 모델 가중치가 저장되었습니다.
상세 분석 결과가 저장되었습니다: ./infrastructure/models/weights/checkpoints/emonet_1_percent_trained_metrics.json
🏃 View run welcoming-fish-385 at: http://127.0.0.1:5000/#/experiments/957529077469630842/runs/11132a1dd7a448c1910caa353951045d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/957529077469630842


[I 2025-08-23 00:29:21,598] Trial 4 finished with value: 1.2301 and parameters: {'lr': 7.470764901176699e-05, 'optimizer': 'AdamW', 'scheduler': 'ReduceLROnPlateau'}. Best is trial 4 with value: 1.2301.


Hyperparameter optimization finished.
Best trial:
  Value (Best Val Loss): 1.2301
  Params: 
    lr: 7.470764901176699e-05
    optimizer: AdamW
    scheduler: ReduceLROnPlateau


: 

In [ ]:
# 데이터 양을 늘려도 성능이 특정 수준에서 다시 정체된다면, 파인튜닝 세분화를 적용하여 모델의 학습 효율을 극대화
# 새로 학습시킬 파라미터와 미세 조정할 파라미터를 분리
new_classifier_params = model.emo_fc_3.parameters()
pretrained_params = [p for name, p in model.named_parameters() if 'emo_fc_3' not in name]

optimizer = optim.Adam([
    {'params': pretrained_params, 'lr': LEARNING_RATE * 0.1}, # 기존 부분은 10분의 1로 미세 조정
    {'params': new_classifier_params, 'lr': LEARNING_RATE}      # 새 부분은 원래 학습률로 학습
], weight_decay=1e-4)